## GPX篩選特定坐標Python程式說明

### 基本要求
- 找出gpx文件中，time欄位時間差>300s，且Speed欄位平均數小於1.8的坐標
- 把符合條件的區間點，都保存起來

### 實現思路
- 遍歷文件夾，獲取所有gpx文件路徑
- 依次訪問gpx文件（excel格式）
- 處理Duration及Speed單位（去掉單位符號：時間為sec，速度為kph）
- 設置window概念，每次向下移動1步，獲取window區間內符合基本要求的點
- 將符合的點重新生成gpx文件，輸出格式為excel

### 相關公式
- 平均速度的計算：<font color=red>區間總距離➗區間總時間</font>
$$ avgSpeed = \sum_{i}\frac{\frac{Duration_i}{3600} * Speed_i}{{\frac{\sum_i{Duration_i}}{3600}}}$$

### 特殊狀況
- 問題1：相鄰window，篩選出同一個點
    - 可以只选取第一次满足以前的点，剩下的不保存
- 問題2：一個window中，選取多少個點<font color=red>(同1, 影響window的區間設置)</font>
- 問題3：gpx文件的時間格式包含三種情況:
    - 1、僅有sec
    - 2、僅有min
    - 3、同時包含sec與min
- 問題4：缺失值的處理，填充0 還是mean

In [ ]:
import pandas as pd
import warnings
import re
import os

warnings.filterwarnings('ignore')

In [ ]:
#读取所有文件路径
def readFolder(filePath, savePath):
    '''
    输入文件路径，返回该路径folder下所有文件的绝对路径
    ex. filePath = '/Users/vincentyau/Desktop/test-01/'
        path = readFolder(filePath)
    '''
    pathDir = os.listdir(filePath)
    new_path_list = []
    save_path_list = []
    for path in pathDir:
        #Mac 下会产生的文件
        if path == '.DS_Store':
            continue
        child = os.path.join('{}{}'.format(filePath, path))
        saveChild = os.path.join('{}{}'.format(savePath, path))
        new_path_list.append(child)
        save_path_list.append(saveChild)
    return new_path_list, save_path_list

#读取文档数据，并返回data
def readData(gpxFilePath):
    data = pd.read_excel(gpxFilePath)
    data.fillna(0, inplace=True)
    return data

#格式化Duration和Speed的单位
def formatData(data):
    '''
    Format Duration and Speed Notation
    返回 处理后的data
    res = re.findall('[0-9]{1,3}', duration_string)
    data['Duration'][i] = float(res[0])*60
    res = re.findall('[0-9]{1,3}', duration) #这边的匹配是1-3个数字格式
    data['Duration'][i] = res[0] #正则findall返回的是单元素list，故采用[0]的方式，但可能要考虑有多个元素的list
    '''
    length_duration = len(data['Duration']) # length_duration = 1566

    #这边处理时间格式，会存在min或者sec的可能，但要预留后面可能存在更复杂的格式问题
    for i in range(length_duration):
        duration = data['Duration'][i]
        duration_string = str(duration)
        
        #这边没必要这么写，只要把原单位去掉，直接获取对应的数字就可以了，可使用strip()与int()
        if 'sec' in duration_string:
            res = float(duration_string.replace('sec', '').strip())
            data['Duration'][i] = res

        elif 'min' in duration_string:
            res = float(duration_string.replace('min', '').strip())
            data['Duration'][i] = res*60

        speed = data['Speed'][i]
        data['Speed'][i] = str(speed).replace('kph', '') #使用replace来加快运行的速度（re的开销太大）
        
    return data

#返回的点为符合的部分，但是现在有一个运行速度的问题，处理速度太慢了
def pointFileter(data):
    '''
    过滤gpx点数据，返回符合规格的点
    '''
    total_time = 0.0
    total_speed = 0.0
    total_distance = 0.0
    sliding_window = 400 #后续可根据平均时间做计算，取整
    points_list = [] #保存被筛选出来的点

    # #遍历一遍原数据的所有点
    length_duration = len(data['Duration'])
    for i in range(length_duration):

        #从第i个元素开始，用window往下滑来获取数据
        index_range = i + sliding_window
        window_list = []
        data_slice = data[i:index_range]
        
        #这里有个很大的提高点，可以将下面的切片先计算完
        for index, row in data_slice.iterrows():
            current_duration = float(row.Duration)
            current_speed = float(row.Speed)
            
            total_time = total_time + current_duration #累积时间，以sec计算
            total_distance = (current_duration/3600) * current_speed + total_distance #累积距离kph = sum(current_duration * current_speed)，(sec/3600) * kph

            #记录每一个点
            window_list.append(row)
            if total_time >= 300:
                avg_speed = float(total_distance / (total_time/3600))
                if avg_speed < 1.8:  #这边需要添加一个判断，如果平均速度不超过某个值，則把前面所有的点都加起来
                    points_list = points_list + window_list

                #需要计算完所有的吗: 一超过300我就停止计算该window的剩余内容, 也就是break回去
                total_distance = 0.0
                total_time = 0.0
                total_speed = 0.0
                break
        total_distance = 0.0
        total_time = 0.0
        total_speed = 0.0
    
    #这里报了一个错：keyerror，但是不去重对后续处理影响不大，所以这里这样处理（这个错是因为数据为空造成的）
    try:
        new_data = pd.DataFrame(points_list).drop_duplicates('Number', 'first', inplace=False)
    except Exception as e:
        new_data = pd.DataFrame(points_list)
        print('这里有一个exception:{}'.format(e))
        
    print('\033[34m符合条件的点共找到{}条\033[0m'.format(len(new_data)))
    
    return new_data

#写入数据到指定的文件
def writeDataToExcel(new_data, fileName):
    new_data.to_excel(fileName)
    print('成功写入文件:{}'.format(fileName))
    
def main(filePath):
    savePath = filePath[:-1] + 'new/'
    if not os.path.exists(savePath):
        os.makedirs(savePath) 

    #这边有mac的某个文件在，需要去掉，
    new_path_list, save_path_list = readFolder(filePath, savePath)
    file_length = len(new_path_list)

    for item in range(file_length):
        
        file = new_path_list[item]
        print('-----现在整顿第{}条数据:{}-----'.format(item+1, file))
        save_file = save_path_list[item]
        try:
            data = readData(file)
            print('处理它的时间、速度格式中....')
            format_Data = formatData(data)
            print('正在为你过滤所需要的点哦....')
            new_data = pointFileter(format_Data)
        except Exception as e:
            print('这里有一条错了，但跳过吧：{}'.format(e))
            continue
        #print('正在完成第{}条数据'.format(item+1))
        writeDataToExcel(new_data, save_file)
        os.remove(file)
        print('已经删除刚处理好的文件{}'.format(file))

In [ ]:
filePath = '/Users/vincentyau/Desktop/501-700/'
main(filePath)

----